## Recommendations with MovieTweetings: Collaborative Filtering

One of the most popular methods for making recommendations is **collaborative filtering**.  In collaborative filtering, you are using the collaboration of user-item recommendations to assist in making new recommendations.  

There are two main methods of performing collaborative filtering:

1. **Neighborhood-Based Collaborative Filtering**, which is based on the idea that we can either correlate items that are similar to provide recommendations or we can correlate users to one another to provide recommendations.

2. **Model Based Collaborative Filtering**, which is based on the idea that we can use machine learning and other mathematical models to understand the relationships that exist amongst items and users to predict ratings and provide ratings.


In this notebook, you will be working on performing **neighborhood-based collaborative filtering**.  There are two main methods for performing collaborative filtering:

1. **User-based collaborative filtering:** In this type of recommendation, users related to the user you would like to make recommendations for are used to create a recommendation.

2. **Item-based collaborative filtering:** In this type of recommendation, first you need to find the items that are most related to each other item (based on similar ratings).  Then you can use the ratings of an individual on those similar items to understand if a user will like the new item.

In this notebook you will be implementing **user-based collaborative filtering**.  However, it is easy to extend this approach to make recommendations using **item-based collaborative filtering**.  First, let's read in our data and necessary libraries.

**NOTE**: Because of the size of the datasets, some of your code cells here will take a while to execute, so be patient!

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t
from scipy.sparse import csr_matrix
from IPython.display import HTML


%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

print(reviews.head())

   user_id  movie_id  rating   timestamp                 date  month_1  \
0        1     68646      10  1381620027  2013-10-12 23:20:27        0   
1        1    113277      10  1379466669  2013-09-18 01:11:09        0   
2        2    422720       8  1412178746  2014-10-01 15:52:26        0   
3        2    454876       8  1394818630  2014-03-14 17:37:10        0   
4        2    790636       7  1389963947  2014-01-17 13:05:47        0   

   month_2  month_3  month_4  month_5  ...  month_9  month_10  month_11  \
0        0        0        0        0  ...        0         1         0   
1        0        0        0        0  ...        0         0         0   
2        0        0        0        0  ...        0         1         0   
3        0        0        0        0  ...        0         0         0   
4        0        0        0        0  ...        0         0         0   

   month_12  year_2013  year_2014  year_2015  year_2016  year_2017  year_2018  
0         0          1  

### Measures of Similarity

When using **neighborhood** based collaborative filtering, it is important to understand how to measure the similarity of users or items to one another.  

There are a number of ways in which we might measure the similarity between two vectors (which might be two users or two items).  In this notebook, we will look specifically at two measures used to compare vectors:

* **Pearson's correlation coefficient**

Pearson's correlation coefficient is a measure of the strength and direction of a linear relationship. The value for this coefficient is a value between -1 and 1 where -1 indicates a strong, negative linear relationship and 1 indicates a strong, positive linear relationship. 

If we have two vectors x and y, we can define the correlation between the vectors as:


$$CORR(x, y) = \frac{\text{COV}(x, y)}{\text{STDEV}(x)\text{ }\text{STDEV}(y)}$$

where 

$$\text{STDEV}(x) = \sqrt{\frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})^2}$$

and 

$$\text{COV}(x, y) = \frac{1}{n-1}\sum_{i=1}^{n}(x_i - \bar{x})(y_i - \bar{y})$$

where n is the length of the vector, which must be the same for both x and y and $\bar{x}$ is the mean of the observations in the vector.  

We can use the correlation coefficient to indicate how alike two vectors are to one another, where the closer to 1 the coefficient, the more alike the vectors are to one another.  There are some potential downsides to using this metric as a measure of similarity.  You will see some of these throughout this workbook.


* **Euclidean distance**

Euclidean distance is a measure of the straightline distance from one vector to another.  Because this is a measure of distance, larger values are an indication that two vectors are different from one another (which is different than Pearson's correlation coefficient).

Specifically, the euclidean distance between two vectors x and y is measured as:

$$ \text{EUCL}(x, y) = \sqrt{\sum_{i=1}^{n}(x_i - y_i)^2}$$

Different from the correlation coefficient, no scaling is performed in the denominator.  Therefore, you need to make sure all of your data are on the same scale when using this metric.

**Note:** Because measuring similarity is often based on looking at the distance between vectors, it is important in these cases to scale your data or to have all data be in the same scale.  If some measures are on a 5 point scale, while others are on a 100 point scale, you are likely to have non-optimal results due to the difference in variability of your features.  In this case, we will not need to scale data because they are all on a 10 point scale, but it is always something to keep in mind!

------------

### User-Item Matrix

In order to calculate the similarities, it is common to put values in a matrix.  In this matrix, users are identified by each row, and items are represented by columns.  


![alt text](imgs/userxitem.png "User Item Matrix")


In the above matrix, you can see that **User 1** and **User 2** both used **Item 1**, and **User 2**, **User 3**, and **User 4** all used **Item 2**.  However, there are also a large number of missing values in the matrix for users who haven't used a particular item.  A matrix with many missing values (like the one above) is considered **sparse**.

Our first goal for this notebook is to create the above matrix with the **reviews** dataset.  However, instead of 1 values in each cell, you should have the actual rating.  

The users will indicate the rows, and the movies will exist across the columns. To create the user-item matrix, we only need the first three columns of the **reviews** dataframe, which you can see by running the cell below.

In [32]:
user_items = reviews[['user_id', 'movie_id', 'rating']]
user_items=user_items.iloc[:10000,:]#.sample(10000)
user_items.head()


,user_id,movie_id,rating
0,1,68646,10
1,1,113277,10
2,2,422720,8
3,2,454876,8
4,2,790636,7


### Creating the User-Item Matrix

In order to create the user-items matrix (like the one above), I personally started by using a [pivot table](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.pivot_table.html). 

However, I quickly ran into a memory error (a common theme throughout this notebook).  I will help you navigate around many of the errors I had, and achieve useful collaborative filtering results! 

_____

`1.` Create a matrix where the users are the rows, the movies are the columns, and the ratings exist in each cell, or a NaN exists in cells where a user hasn't rated a particular movie. If you get a memory error (like I did), [this link here](https://stackoverflow.com/questions/39648991/pandas-dataframe-pivot-memory-error) might help you!

In [33]:
# Create user-by-item matrix
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

In [ ]:
#user_by_movie.to_pickle('user_by_movie.p')

In [34]:
user_by_movie

movie_id,417,12349,13427,15002,15324,15864,21576,21749,22100,22279,...,7068818,7137846,7214842,7291268,7357302,7451284,7608028,7608418,7608534,7843946
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Check your results below to make sure your matrix is ready for the upcoming sections.

In [8]:
assert movies.shape[0] == user_by_movie.shape[1], "Oh no! Your matrix should have {} columns, and yours has {}!".format(movies.shape[0], user_by_movie.shape[1])
assert reviews.user_id.nunique() == user_by_movie.shape[0], "Oh no! Your matrix should have {} rows, and yours has {}!".format(reviews.user_id.nunique(), user_by_movie.shape[0])
print("Looks like you are all set! Proceed!")
HTML('<img src="images/greatjob.webp">')

Looks like you are all set! Proceed!


`2.` Now that you have a matrix of users by movies, use this matrix to create a dictionary where the key is each user and the value is an array of the movies each user has rated.

In [40]:
# Create a dictionary with users and corresponding movies seen

def movies_watched(user_id):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    OUTPUT:
    movies - an array of movies the user has watched
    '''
    movies = user_by_movie.loc[user_id].dropna().index.values.tolist()
    return movies


def create_user_movie_dict():
    '''
    INPUT: None
    OUTPUT: movies_seen - a dictionary where each key is a user_id and the value is an array of movie_ids
    
    Creates the movies_seen dictionary
    '''    
    movies_seen = dict()
    num_usrs=user_by_movie.shape[0]
    for user in range(1, num_usrs+1):
        movies_seen[user] = movies_watched(user)    
    return movies_seen


# Use your function to return dictionary
movies_seen = create_user_movie_dict()

In [41]:
movies_seen

{1: [68646, 113277],
 2: [422720,
  454876,
  790636,
  816711,
  1091191,
  1103275,
  1322269,
  1390411,
  1398426,
  1431045,
  1433811,
  1454468,
  1535109,
  1675434,
  1798709,
  2017038,
  2024544,
  2294629,
  2361509,
  2381249,
  2726560,
  2883512,
  3079380],
 3: [1790864, 2170439, 2203939],
 4: [1300854],
 5: [54953, 120863],
 6: [2103281],
 7: [1764234, 1790885, 2053463],
 8: [385002, 1220198, 1462900, 1512685, 1631707, 1986994, 1999995],
 9: [65207, 363163, 985699],
 10: [1253863],
 11: [3294634],
 12: [1255953],
 13: [2649554],
 14: [452623],
 15: [1980929],
 16: [800241],
 17: [68646, 71562, 99674],
 18: [2258858],
 19: [390384],
 20: [1549589, 1605783],
 21: [4094724],
 22: [359950,
  770828,
  831387,
  1091191,
  1170358,
  1210819,
  1430132,
  1490017,
  1535108,
  1663662,
  1843866,
  1872181,
  2103281,
  2179136,
  2209764,
  2267998,
  2333784,
  2334879,
  2713180,
  2872732],
 23: [2674426],
 24: [816692, 2245084, 2267998, 2488496, 2802144, 2820852, 33005

`3.` If a user hasn't rated more than 2 movies, we consider these users "too new".  Create a new dictionary that only contains users who have rated more than 2 movies.  This dictionary will be used for all the final steps of this workbook.

In [45]:
# Remove individuals who have watched 2 or fewer movies - don't have enough data to make recs

def create_movies_to_analyze(movies_seen, lower_bound=2):
    '''
    INPUT:  
    movies_seen - a dictionary where each key is a user_id and the value is an array of movie_ids
    lower_bound - (an int) a user must have more movies seen than the lower bound to be added to the movies_to_analyze dictionary

    OUTPUT: 
    movies_to_analyze - a dictionary where each key is a user_id and the value is an array of movie_ids
    
    The movies_seen and movies_to_analyze dictionaries should be the same except that the output dictionary has removed 
    
    '''    
    # Do things to create updated dictionary
    movies_to_analyze = dict()
    for user, movies in movies_seen.items():
        if len(movies) > lower_bound:
            movies_to_analyze[user] = movies
    
    return movies_to_analyze


# Use your function to return your updated dictionary
movies_to_analyze = create_movies_to_analyze(movies_seen)

In [47]:
user_by_movie.shape[0]#before

876

In [49]:
len(movies_to_analyze.keys())#after

369

In [ ]:
# Run the tests below to check that your movies_to_analyze matches the solution
assert len(movies_to_analyze) == 23512, "Oops!  It doesn't look like your dictionary has the right number of individuals."
assert len(movies_to_analyze[2]) == 23, "Oops!  User 2 didn't match the number of movies we thought they would have."
assert len(movies_to_analyze[7])  == 3, "Oops!  User 7 didn't match the number of movies we thought they would have."
print("If this is all you see, you are good to go!")

### Calculating User Similarities

Now that you have set up the **movies_to_analyze** dictionary, it is time to take a closer look at the similarities between users.  Below is the pseudocode for how I thought about determining the similarity between users:

```
for user1 in movies_to_analyze
    for user2 in movies_to_analyze
        see how many movies match between the two users
        if more than two movies in common
            pull the overlapping movies
            compute the distance/similarity metric between ratings on the same movies for the two users
            store the users and the distance metric
```

However, this took a very long time to run, and other methods of performing these operations did not fit on the workspace memory!

Therefore, rather than creating a dataframe with all possible pairings of users in our data, your task for this question is to look at a few specific examples of the correlation between ratings given by two users.  For this question consider you want to compute the [correlation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) between users.

`4.` Using the **movies_to_analyze** dictionary and **user_by_movie** dataframe, create a function that computes the correlation between the ratings of similar movies for two users.  Then use your function to compare your results to ours using the tests below.  

In [62]:
def compute_correlation(user1, user2):
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    OUTPUT
    the correlation between the matching ratings between the two users
    '''  
    movies_1 = movies_to_analyze[user1]
    movies_2 = movies_to_analyze[user2]
    
    common_movies = np.intersect1d(movies_1, movies_2, assume_unique=True)
    corr = user_by_movie.loc[[user1, user2], common_movies].T.corr().iloc[0, 1]#off diagonal value
    
    return corr #return the correlation #person correlation is the adjusted similarity there

In [63]:
compute_correlation(2, 104)

nan

In [64]:
compute_correlation(2,2)

1.0

In [65]:
round(compute_correlation(2,66), 2)

0.76

In [66]:
np.isnan(compute_correlation(2,104))

True

In [67]:
# Test your function against the solution
assert compute_correlation(2,2) == 1.0, "Oops!  The correlation between a user and itself should be 1.0."
assert round(compute_correlation(2,66), 2) == 0.76, "Oops!  The correlation between user 2 and 66 should be about 0.76."
assert np.isnan(compute_correlation(2,104)), "Oops!  The correlation between user 2 and 104 should be a NaN."

print("If this is all you see, then it looks like your function passed all of our tests!")

If this is all you see, then it looks like your function passed all of our tests!


### Why the NaN's?

If the function you wrote passed all of the tests, then you have correctly set up your function to calculate the correlation between any two users.  

`5.` But one question is, why are we still obtaining **NaN** values?  As you can see in the code cell above, users 2 and 104 have a correlation of **NaN**. Why?

Think and write your ideas here about why these NaNs exist, and use the cells below to do some coding to validate your thoughts. You can check other pairs of users and see that there are actually many NaNs in our data - 2,526,710 of them in fact. These NaN's ultimately make the correlation coefficient a less than optimal measure of similarity between two users.




In [52]:
# Which movies did both user 2 and user 104 see?
movies_1 = movies_to_analyze[2]
movies_2 = movies_to_analyze[104]
common_movies = np.intersect1d(movies_1, movies_2, assume_unique=True)
common_movies

array([ 454876,  816711, 1454468, 1535109])

In [68]:
# What were the ratings for each user for those movies?
user_by_movie.loc[[2, 104], common_movies]

movie_id,454876,816711,1454468,1535109
user_id,,,,
2,8.0,8.0,8.0,8.0
104,9.0,7.0,7.0,9.0


`6.` Because the correlation coefficient proved to be less than optimal for relating user ratings to one another, we could instead calculate the euclidean distance between the ratings.  I found [this post](https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy) particularly helpful when I was setting up my function.  This function should be very similar to your previous function.  When you feel confident with your function, test it against our results.

In [79]:
# from scipy.spatial import distance
def compute_euclidean_dist(user1, user2):
    '''
    INPUT
    user1 - int user_id
    user2 - int user_id
    OUTPUT
    the euclidean distance between user1 and user2
    '''
    movies_1 = movies_to_analyze[user1]
    movies_2 = movies_to_analyze[user2]
    
    common_movies = np.intersect1d(movies_1, movies_2, assume_unique=True)
    
#     v1=np.array(user_by_movie.loc[user1,common_movies])
#     v2=np.array(user_by_movie.loc[user2,common_movies])
#     dist = distance.euclidean(v1, v2)
   
    intersection = user_by_movie.loc[[user1, user2], common_movies]
    dist = np.linalg.norm(intersection.loc[user1] - intersection.loc[user2], ord=2)
    
#     if len(common_movies) > 2:
#         a = np.array(user_by_movie.loc[user1, common_movies])
#         b = np.array(user_by_movie.loc[user2, common_movies])
#         dist = np.sqrt(np.sum(np.square(a-b)))
    
    return dist #return the euclidean distance

In [80]:
compute_euclidean_dist(2, 104)

2.0

In [90]:
compute_euclidean_dist(2, 2)

0.0

In [82]:
round(compute_euclidean_dist(54,44), 2)

5.1

In [83]:
# Test your function against the solution
assert compute_euclidean_dist(2,2) == 0, "Oops! Wrong"
assert round(compute_euclidean_dist(54,44), 2) == 5.1, "Oops! Wrong"
assert np.isnan(compute_euclidean_dist(2,104)) == False, "Oops! Wrong"

print("If this is all you see, then it looks like your function passed all of our tests!")

If this is all you see, then it looks like your function passed all of our tests!


In [91]:
# Read in solution euclidean distances"
import pickle
df_dists = pd.read_pickle("dists.p")

In [92]:
df_dists.shape

(31813482, 3)

In [93]:
df_dists = df_dists.iloc[:20000, :]
df_dists.shape

(20000, 3)

In [94]:
df_dists.head()

,user1,user2,eucl_dist
0,2,2,0.000000
1,2,66,2.236068
2,2,90,5.385165
3,2,99,2.828427
4,2,104,2.000000


In [98]:
d = pd.DataFrame([])

for user1 in movies_to_analyze:#in [2, 17, 66, 90, 99, 104]
    for user2 in movies_to_analyze:# in [2, 17, 66, 90, 99, 104]
        if len(np.intersect1d(movies_to_analyze[user1], movies_to_analyze[user2], assume_unique=True)) != 0:# >=2
            if(user1 == user2):
                d = d.append([{'user1': user1, 'user2': user2, 'dist':0}], ignore_index=True)
            else:
                d = d.append([{'user1': user1, 'user2': user2, 'dist':compute_euclidean_dist(user1, user2)}], ignore_index=True)

In [108]:
#version 1
import time
start = time.time()

d = pd.DataFrame([])
for user1 in [2, 17, 66, 90, 99, 104]:
    for user2 in [2, 17, 66, 90, 99, 104]:
        if len(np.intersect1d(movies_to_analyze[user1], movies_to_analyze[user2], assume_unique=True)) != 0:
            d = d.append([{'user1': user1, 'user2': user2, 'dist':compute_euclidean_dist(user1, user2)}], ignore_index=True)
                
end = time.time()
print((end-start))

0.20600080490112305


In [105]:
#version 2
import time
start = time.time()

d = pd.DataFrame([])
for user1 in [2, 17, 66, 90, 99, 104]:
    for user2 in [2, 17, 66, 90, 99, 104]:
        if len(np.intersect1d(movies_to_analyze[user1], movies_to_analyze[user2], assume_unique=True)) != 0:# 
            if(user1 == user2):
                d = d.append([{'user1': user1, 'user2': user2, 'dist':0}], ignore_index=True)
            else:
                d = d.append([{'user1': user1, 'user2': user2, 'dist':compute_euclidean_dist(user1, user2)}], ignore_index=True)
                
end = time.time()
print((end-start))

0.12395191192626953


In [107]:
#version 3
import time
start = time.time()

d = pd.DataFrame([])
for user1 in [2, 17, 66, 90, 99, 104]:
    for user2 in [2, 17, 66, 90, 99, 104]:
        if len(np.intersect1d(movies_to_analyze[user1], movies_to_analyze[user2], assume_unique=True)) != 0:
            if(user1 == user2):
                d = d.append([{'user1': user1, 'user2': user2, 'dist':0}], ignore_index=True)
                
            elif len(np.where((d['user1']==user1) & (d['user2']==user2))[0]) == 0:
                eucl_dist = compute_euclidean_dist(user1, user2)
                d = d.append([{'user1': user1, 'user2': user2, 'dist':eucl_dist}], ignore_index=True)
                d = d.append([{'user1': user2, 'user2': user1, 'dist':eucl_dist}], ignore_index=True)

end = time.time()
print((end-start))

0.09794139862060547


In [103]:
d.head()

,user1,user2,dist
0,2,2,0.000000
1,2,66,2.236068
2,2,90,5.385165
3,2,99,2.828427
4,2,104,2.000000


### Using the Nearest Neighbors to Make Recommendations

In the previous question, you read in **df_dists**. Therefore, you have a measure of distance between each user and every other user. This dataframe holds every possible pairing of users, as well as the corresponding euclidean distance.

Because of the **NaN** values that exist within the correlations of the matching ratings for many pairs of users, as we discussed above, we will proceed using **df_dists**. You will want to find the users that are 'nearest' each user.  Then you will want to find the movies the closest neighbors have liked to recommend to each user.

I made use of the following objects:

* df_dists (to obtain the neighbors)
* user_items (to obtain the movies the neighbors and users have rated)
* movies (to obtain the names of the movies)

`7.` Complete the functions below, which allow you to find the recommendations for any user.  There are five functions which you will need:

* **find_closest_neighbors** - this returns a list of user_ids from closest neighbor to farthest neighbor using euclidean distance


* **movies_liked** - returns an array of movie_ids


* **movie_names** - takes the output of movies_liked and returns a list of movie names associated with the movie_ids


* **make_recommendations** - takes a user id and goes through closest neighbors to return a list of movie names as recommendations


* **all_recommendations** = loops through every user and returns a dictionary of with the key as a user_id and the value as a list of movie recommendations

In [118]:
np.array(df_dists[df_dists['user1']==2].sort_values(by='eucl_dist').iloc[1:]['user2']).tolist()

[755,
 22915,
 35310,
 34706,
 33207,
 9356,
 8944,
 12856,
 32951,
 44688,
 20390,
 39336,
 1885,
 53793,
 2138,
 30884,
 39417,
 25270,
 26605,
 26585,
 38636,
 2023,
 14774,
 32575,
 14494,
 12284,
 38932,
 25961,
 45808,
 46415,
 39265,
 13215,
 52049,
 13298,
 25620,
 2341,
 39414,
 26103,
 26905,
 10082,
 38101,
 50988,
 4537,
 4935,
 35699,
 31200,
 5386,
 50681,
 4241,
 30845,
 5746,
 30744,
 6025,
 51282,
 35479,
 6456,
 36307,
 7292,
 27267,
 15113,
 48100,
 37711,
 3368,
 37526,
 47664,
 9393,
 8782,
 51795,
 8401,
 8266,
 31732,
 7519,
 8948,
 24290,
 2548,
 51137,
 44210,
 53474,
 34726,
 21660,
 17974,
 33156,
 16280,
 43001,
 16148,
 42293,
 43797,
 23909,
 42858,
 21223,
 18557,
 42780,
 43345,
 43890,
 33316,
 21888,
 18192,
 20610,
 53439,
 19514,
 49437,
 36462,
 21552,
 19283,
 8268,
 29086,
 19113,
 19264,
 8504,
 49127,
 28453,
 43559,
 48834,
 28198,
 37421,
 37435,
 18934,
 18566,
 9597,
 49274,
 7379,
 49977,
 36335,
 20894,
 4820,
 42781,
 21152,
 50795,
 3099

In [121]:
find_closest_neighbors(2)

array([  755, 22915, 35310, ..., 36807, 32494, 52737], dtype=int64)

In [126]:
np.array(user_items.query("user_id == 2 and rating > 7")['movie_id'])# 7 is mythreshold assumption

array([ 422720,  454876,  816711, 1390411, 1398426, 1433811, 1454468,
       1535109, 1675434, 1798709, 2024544, 2294629, 2361509, 2726560,
       2883512, 3079380], dtype=int64)

In [132]:
movies_liked(2, min_rating=7)

array([ 422720,  454876,  816711, 1390411, 1398426, 1433811, 1454468,
       1535109, 1675434, 1798709, 2024544, 2294629, 2361509, 2726560,
       2883512, 3079380], dtype=int64)

In [138]:
movie_names([422720,  454876,  816711])

['Marie Antoinette (2006)', 'Life of Pi (2012)', 'World War Z (2013)']

In [146]:
movies_watched(2)

[422720,
 454876,
 790636,
 816711,
 1091191,
 1103275,
 1322269,
 1390411,
 1398426,
 1431045,
 1433811,
 1454468,
 1535109,
 1675434,
 1798709,
 2017038,
 2024544,
 2294629,
 2361509,
 2381249,
 2726560,
 2883512,
 3079380]

In [165]:
make_recommendations(2)

[['Silver Linings Playbook (2012)',
  '50/50 (2011)',
  'Gravity (2013)',
  'Captain Phillips (2013)',
  'The Perks of Being a Wallflower (2012)',
  'Django Unchained (2012)',
  'Gone Girl (2014)',
  'Nightcrawler (2014)'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [158]:
movies_to_analyze.keys()

dict_keys([2, 3, 7, 8, 9, 17, 22, 24, 25, 26, 30, 31, 32, 33, 34, 39, 42, 43, 44, 45, 48, 51, 52, 53, 54, 55, 57, 61, 64, 66, 71, 73, 74, 75, 77, 79, 85, 86, 89, 90, 96, 97, 99, 101, 102, 104, 106, 108, 109, 112, 119, 121, 122, 124, 125, 129, 130, 135, 136, 138, 139, 140, 141, 146, 147, 149, 152, 155, 156, 157, 158, 159, 162, 164, 166, 171, 174, 175, 180, 181, 183, 187, 190, 199, 200, 201, 202, 206, 207, 208, 209, 212, 214, 219, 221, 224, 225, 227, 228, 230, 235, 238, 240, 242, 246, 251, 257, 258, 259, 260, 261, 262, 265, 266, 268, 270, 272, 273, 274, 275, 277, 287, 288, 290, 291, 292, 293, 294, 295, 296, 298, 304, 306, 307, 308, 309, 310, 313, 314, 315, 319, 321, 328, 331, 332, 336, 338, 341, 342, 344, 347, 348, 349, 350, 354, 355, 359, 360, 362, 369, 376, 388, 389, 392, 393, 396, 408, 412, 413, 414, 415, 417, 418, 420, 424, 427, 428, 431, 434, 436, 440, 441, 443, 445, 447, 449, 452, 453, 454, 458, 459, 460, 461, 463, 464, 466, 469, 470, 471, 472, 482, 484, 485, 486, 487, 489, 490, 49

In [168]:
def find_closest_neighbors(user):
    '''
    INPUT:
        user - (int) the user_id of the individual you want to find the closest users
    OUTPUT:
        closest_neighbors - an array of the id's of the users sorted from closest to farthest away
    '''
    # I treated ties as arbitrary and just kept whichever was easiest to keep using the head method
    # You might choose to do something less hand wavy - order the neighbors
    closest_neighbors = np.array(df_dists[df_dists['user1'] == user].sort_values(by='eucl_dist').iloc[1:]['user2'])
    return closest_neighbors
    
    
    
def movies_liked(user_id, min_rating=7):
    '''
    INPUT:
    user_id - the user_id of an individual as int
    min_rating - the minimum rating considered while still a movie is still a "like" and not a "dislike"
    OUTPUT:
    movies_liked - an array of movies the user has watched and liked
    '''
    movies_liked = np.array(user_items.query("user_id == @user_id and rating > @min_rating")['movie_id'])
    return movies_liked


def movie_names(movie_ids):
    '''
    INPUT
    movie_ids - a list of movie_ids
    OUTPUT
    movies - a list of movie names associated with the movie_ids
    
    '''
    movie_lst = list(movies[movies['movie_id'].isin(movie_ids)]['movie'])
    return movie_lst
    
    
def make_recommendations(user, num_recs=10):
    '''
    INPUT:
        user - (int) a user_id of the individual you want to make recommendations for
        num_recs - (int) number of movies to return
    OUTPUT:
        recommendations - a list of movies - if there are "num_recs" recommendations return this many
                          otherwise return the total number of recommendations available for the "user"
                          which may just be an empty list
    '''
    recommendations = []
    movie_ids = []
    movies_seen = movies_watched(user)
    closest_neighbors = find_closest_neighbors(user)
    # go through the neighbors and identify movies they like and the user hasn't seen
    for i in closest_neighbors:
        #obtain recommendations for each neighbor        
        movie_ids.append(movies_liked(i))
                
    for j in movie_ids:
        if movie_ids not in movies_seen:
            #store recommendations
            recommendations.append(movie_names(j))
    recommendations = recommendations[:num_recs]
    #pull movie titles not user ids
    return recommendations

def all_recommendations(num_recs=10):
    '''
    INPUT 
        num_recs (int) the (max) number of recommendations for each user
    OUTPUT
        all_recs - a dictionary where each key is a user_id and the value is an array of recommended movie titles
    '''
    
    all_recs = dict()
    # Make the recommendations for each user
    all_users = len(movies_to_analyze.keys())
    for i in range(1, all_users+1):
        all_recs[i] = make_recommendations(i, num_recs)

    return all_recs


In [169]:
all_recs = all_recommendations(10)
all_recs # i took chunk of 20000 record of 31813482 records dur to memory issues :D

{1: [],
 2: [['Silver Linings Playbook (2012)',
   '50/50 (2011)',
   'Gravity (2013)',
   'Captain Phillips (2013)',
   'The Perks of Being a Wallflower (2012)',
   'Django Unchained (2012)',
   'Gone Girl (2014)',
   'Nightcrawler (2014)'],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 3: [[], [], [], [], [], [], [], [], [], []],
 4: [],
 5: [],
 6: [],
 7: [[],
  ['Minority Report (2002)',
   'The Bourne Identity (2002)',
   'The Chronicles of Riddick (2004)',
   'The Bourne Supremacy (2004)',
   'Batman Begins (2005)',
   'Spider-Man 3 (2007)',
   'Transformers (2007)',
   'The Bourne Ultimatum (2007)',
   'Body of Lies (2008)',
   'Drive (2011)',
   'Jack Reacher (2012)',
   'Star Trek (2009)',
   'Interstellar (2014)',
   'World War Z (2013)',
   'The Avengers (2012)',
   'The Amazing Spider-Man (2012)',
   'Skyfall (2012)',
   'The Lincoln Lawyer (2011)',
   'The Bourne Legacy (2012)',
   'Mission: Impossible - Ghost Protocol (2011)',
   'Looper (2012)',
   'Iron Man 3

In [3]:
# This loads our solution dictionary so you can compare results
all_recs_sol = pd.read_pickle("all_recs.p")

In [ ]:
assert all_recs[2] == make_recommendations(2), "Oops!  Your recommendations for user 2 didn't match ours."
assert all_recs[26] == make_recommendations(26), "Oops!  It actually wasn't possible to make any recommendations for user 26."
assert all_recs[1503] == make_recommendations(1503), "Oops! Looks like your solution for user 1503 didn't match ours."
print("If you made it here, you now have recommendations for many users using collaborative filtering!")
HTML('<img src="images/greatjob.webp">')

### Now What?

If you made it this far, you have successfully implemented a solution to making recommendations using collaborative filtering. 

`8.` Let's do a quick recap of the steps taken to obtain recommendations using collaborative filtering.  

In [ ]:
# Check your understanding of the results by correctly filling in the dictionary below
a = "pearson's correlation and spearman's correlation"
b = 'item based collaborative filtering'
c = "there were too many ratings to get a stable metric"
d = 'user based collaborative filtering'
e = "euclidean distance and pearson's correlation coefficient"
f = "manhattan distance and euclidean distance"
g = "spearman's correlation and euclidean distance"
h = "the spread in some ratings was zero"
i = 'content based recommendation'

sol_dict = {
    'The type of recommendation system implemented here was a ...': # letter here,
    'The two methods used to estimate user similarity were: ': # letter here,
    'There was an issue with using the correlation coefficient.  What was it?': # letter here
}

t.test_recs(sol_dict)

Additionally, let's take a closer look at some of the results.  There are two solution files that you read in to check your results, and you created these objects

* **df_dists** - a dataframe of user1, user2, euclidean distance between the two users
* **all_recs_sol** - a dictionary of all recommendations (key = user, value = list of recommendations)  

`9.` Use these two objects along with the cells below to correctly fill in the dictionary below and complete this notebook!

In [ ]:
a = 567
b = 1503
c = 1319
d = 1325
e = 2526710
f = 0
g = 'Use another method to make recommendations - content based, knowledge based, or model based collaborative filtering'

sol_dict2 = {
    'For how many pairs of users were we not able to obtain a measure of similarity using correlation?': # letter here,
    'For how many pairs of users were we not able to obtain a measure of similarity using euclidean distance?': # letter here,
    'For how many users were we unable to make any recommendations for using collaborative filtering?': # letter here,
    'For how many users were we unable to make 10 recommendations for using collaborative filtering?': # letter here,
    'What might be a way for us to get 10 recommendations for every user?': # letter here   
}

t.test_recs2(sol_dict2)

In [ ]:
# Use the cells below for any work you need to do!

In [ ]:
# Users without recs


In [ ]:
# NaN euclidean distance values


In [ ]:
# Users with less than 10 recs
